In [ ]:
%matplotlib inline
from deepdecoder.networks import dcgan_generator, dcgan_discriminator, gan_grid_idx, mogan_pyramid
from deepdecoder.grid_curriculum import exam, grids_from_lecture
from deepdecoder.data import normalize_generator, grids_lecture_generator, load_real_hdf5_tags
from keras.optimizers import Adam
from keras.objectives import binary_crossentropy, mse
import numpy as np
import matplotlib.pyplot as plt
import pylab
import time
from itertools import combinations
pylab.rcParams['figure.figsize'] = (16, 8)

In [ ]:
batch_size = 128
num_batches_per_epoch = 100
epoch_size = batch_size*num_batches_per_epoch

generator_input_dim=50

In [ ]:
g = dcgan_generator(n=32, input_dim=generator_input_dim)
d = dcgan_discriminator(n=32)

In [ ]:
g.load_weights("generator_pyramdi_loss.hdf5")

In [ ]:
tags = load_real_hdf5_tags('/home/leon/data/tags.hdf5', batch_size,  num_batches_per_epoch)

In [ ]:
grids_lecture_generator(batch_size)

In [ ]:
mogan = mogan_pyramid(g, d, lambda: Adam(), nb_z=31, gan_objective=mse)

In [ ]:
mogan.mulit_objectives.compile_get_grads()

In [ ]:
mogan.mulit_objectives.grads_positions[0]

In [ ]:
tag_batch = tags[0:batch_size]
params, grid_idx = next(grids_lecture_generator(batch_size))
inputs = [tag_batch.astype(np.float32), grid_idx.astype(np.float32), params.astype(np.float32)]
                                           
grads = mogan.mulit_objectives.get_grads(inputs)

In [ ]:
num_bins = 50
def plt_hist(x, label):
    hist, bins = np.histogram(x, bins=num_bins)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width, label=label, alpha=0.5)

for i, grad_dict in enumerate(grads):
    fig = plt.figure()
    print(mogan.gan.G.params[i])
    print(i)
    for plt_idx, (name, grad) in enumerate(grad_dict.items(), 1):
        fig.add_subplot(3, 1, plt_idx)
        plt.title(name)
        plt_hist(grad, name)
    plt.show()
    print("multiplication")
    for plt_idx, ((name_a, a), (name_b, b)) in enumerate(combinations(grad_dict.items(), 2), 1):
        fig.add_subplot(3, 1, plt_idx)
        plt.title(name)
        plt_hist(a*b, "{}-{}".format(name_a, name_b))
    plt.show()